# Replication of Paper Results

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import csv
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import numpy as np

def create_multi_agent_dynamics_with_center(N, masses, L, controller, V_d, r, theta_d, index_dead, index_second_fault,
                                            k=2.5e4, lambda_i=1, g=9.81, lambda_theta=1):
    """
    """
    assert N >= 1, "There must be at least one agent (the center)."
    assert len(masses) == N

    e3 = np.array([0, 0, 1]).reshape(-1,1)



    def dynamics(t, state):
        dstate_dt = np.zeros_like(state)

        vd = V_d(t)
        u = controller(t, state, vd)

        

        for i in range(0, N):
            idx = 6 * i
            x_i = state[idx : idx + 3]
            v_i = state[idx + 3 : idx + 6]

            m_i = masses[i]
            F_i = u[:,i]
            if flag_dead[0] == True and index_dead == i:
                F_i = np.zeros((3,1))
            if flag_second_fault[0] == True and index_second_fault == i:
                F_i = np.zeros((3,1))

            dx_dt = v_i
            dv_dt = (F_i.reshape(-1,1) - m_i * g * e3) / m_i

            dstate_dt[idx : idx + 3] = dx_dt
            dstate_dt[idx + 3 : idx + 6] = dv_dt.flatten()

        for i in range(6*N, 6*N+N):
            item = i - 6*N
            dMc_i = -lambda_i*(state[item*6+5]-vd[2])+lambda_i*np.sum([state[6*N+j]-state[6*N+item] for j in range(0, N) if L[item, j]!=0])#
            dstate_dt[i] = dMc_i

        for i in range(7*N, 7*N+N):
            item = i - 7*N
            vec = np.zeros((3,1))
            vec[0] = -np.sin((state[i] + theta_d[i-7*N])/2)
            vec[1] = np.cos((state[i] + theta_d[i-7*N])/2)
            #print(state[6*item+3:6*item+6].shape)
            d_theta_i = -lambda_theta*np.sin((state[i]-theta_d[i-7*N])/2) - 2*controller.rho[item]*(state[6*item+3:6*item+6].reshape(-1,1)-vd).T@vec
            dstate_dt[i] = d_theta_i

        
            
        return dstate_dt

    return dynamics

In [ ]:
class Controller():
    def __init__(self, N, Gamma_i, V_d, r, masses, rho_d, g=9.81, log_file='control_log.csv', f_log_file='f_hat_log.csv'):
        self.N = N
        self.Gamma_i = Gamma_i
        self.V_d = V_d
        self.masses = masses
        self.g = g
        self.log_file = log_file
        self.f_log_file = f_log_file
        self.r_norm = r/np.linalg.norm(r, axis=0, keepdims=True)
        self.rho = rho_d
        print(self.r_norm)

        # Create or overwrite the log file and write header
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['time']
            for i in range(N):
                header += [f'u{i}_x', f'u{i}_y', f'u{i}_z']
            writer.writerow(header)
        
        # Create or overwrite the force log file and write header
        with open(self.f_log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            header = ['time']
            for i in range(N):
                header += [f'f{i}_x', f'f{i}_y', f'f{i}_z']
            writer.writerow(header)

    def __call__(self, t, state, vd):
        e3 = np.array([0, 0, 1]).reshape(-1,1)
        out = np.zeros((3, self.N))
        f_log = np.zeros((3, self.N))
        f_coeff = np.array([[0.0, -2.0], [2.0, 1.0], [-2.0, 1.0]])

        
        for i in range(self.N):
            f_hat_i = np.zeros((3,1))
            f_hat_i[0] = (1-np.exp(-t))*np.cos(state[7*N + i])*self.rho[i]
            f_hat_i[1] = (1-np.exp(-t))*np.sin(state[7*N + i])*self.rho[i]
            f_hat_i[2] = (1-np.exp(-t))*state[6*(self.N)+i]*self.g

            f_log[:, i] = f_hat_i.flatten()
            
            out[:,i] = (f_hat_i + self.masses[i]*self.g*e3 - self.Gamma_i @ (state[(i)*6+3:(i)*6+6].reshape(-1,1) - vd)).flatten() 

        # Log control input to CSV
        flat_output = out.T.flatten()  # [u0_x, u0_y, u0_z, u1_x, ...]
        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([t] + flat_output.tolist())
        
        # Log force estimates
        with open(self.f_log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([t] + f_log.T.flatten().tolist())

        return out


In [ ]:
class CircularCloth:
    def __init__(self,
                 radius=2.5,
                 num_rings=10,
                 points_per_ring=30,
                 k=500.0,
                 m=0.1,
                 damping=0.1,
                 gravity=np.array([0, 0, -9.81]),
                 num_beams=3,
                 beam_length=1.5,
                 beam_nodes_per_beam=1,
                 central_index=0):
        
        self.radius = radius
        self.num_rings = num_rings
        self.points_per_ring = points_per_ring
        self.k = k
        self.m = m
        self.damping = damping
        self.gravity = gravity
        self.central_index = central_index
        
        self.num_beams = num_beams
        self.beam_length = beam_length
        self.beam_nodes_per_beam = beam_nodes_per_beam
        
        self.nodes = None
        self.N = 0
        self.edges = []
        self.rest_lengths = None
        self.masses = None
        self.fixed_indices = []
        self.beam_edges = []
        self.state0 = None
        self.adjacency_matrix = None
        
        self._generate_nodes()
        self._generate_edges_and_rest_lengths()
        #self._add_beams()
        self._finalize_state()
        
    def _generate_nodes(self):
        nodes = []
        nodes.append(np.array([0, 0, 0]))  # center point
        for r in range(1, self.num_rings):
            radius_r = self.radius * r / (self.num_rings - 1)
            for i in range(self.points_per_ring):
                angle = 2 * np.pi * i / self.points_per_ring
                x = radius_r * np.cos(angle)
                y = radius_r * np.sin(angle)
                nodes.append(np.array([x, y, 0]))
        self.nodes = np.array(nodes)
        self.N = len(self.nodes)

        # Salva gli indici equispaziati dell'anello più esterno
        start_idx = 1 + (self.num_rings - 2) * self.points_per_ring
        end_idx = start_idx + self.points_per_ring

        # Seleziona N indici equidispaziati tra i nodi dell’anello esterno
        total = self.points_per_ring
        # o self.N_target se definito altrove
        step = total // self.num_beams

        self.fixed_indices = [start_idx + i * step for i in range(self.num_beams)]
        #print(f"Outer ring indices: {self.outer_ring_indices}")

        
    def _get_neighbors(self, idx):
        if idx == 0:
            return list(range(1, self.points_per_ring + 1))
        
        ring_idx = None
        count = 1
        for r in range(1, self.num_rings):
            if idx < count + self.points_per_ring:
                ring_idx = r
                break
            count += self.points_per_ring

        pos_in_ring = idx - (1 + (ring_idx - 1) * self.points_per_ring)

        neighbors = []
        neighbors.append(1 + (ring_idx - 1) * self.points_per_ring + (pos_in_ring - 1) % self.points_per_ring)
        neighbors.append(1 + (ring_idx - 1) * self.points_per_ring + (pos_in_ring + 1) % self.points_per_ring)

        if ring_idx > 1:
            inner_ring_start = 1 + (ring_idx - 2) * self.points_per_ring
            inner_pos = int(pos_in_ring * self.points_per_ring / self.points_per_ring)
            neighbors.append(inner_ring_start + inner_pos % self.points_per_ring)
            neighbors.append(inner_ring_start + (inner_pos - 1) % self.points_per_ring)
        else:
            neighbors.append(0)
        
        if ring_idx < self.num_rings - 1:
            outer_ring_start = 1 + ring_idx * self.points_per_ring
            neighbors.append(outer_ring_start + pos_in_ring % self.points_per_ring)
            neighbors.append(outer_ring_start + (pos_in_ring + 1) % self.points_per_ring)

        neighbors = list(set(n for n in neighbors if 0 <= n < self.N))
        return neighbors
    
    def _generate_edges_and_rest_lengths(self):
        edges = []
        rest_lengths = []
        for i in range(self.N):
            for j in self._get_neighbors(i):
                if i < j:
                    edges.append((i, j))
                    dist = np.linalg.norm(self.nodes[i] - self.nodes[j])
                    rest_lengths.append(dist)
        self.edges = edges
        self.rest_lengths = np.array(rest_lengths)
    
    """def _add_beams(self):
        beam_nodes = []
        beam_edges = []

        beam_angles = np.linspace(0, 2*np.pi, self.num_beams, endpoint=False)
        outer_ring_start = 1 + (self.num_rings - 2) * self.points_per_ring
        outer_ring_end = outer_ring_start + self.points_per_ring

        for angle in beam_angles:
            outer_ring_nodes = self.nodes[outer_ring_start:outer_ring_end]
            target_pos = np.array([self.radius * np.cos(angle), self.radius * np.sin(angle), 0])
            dists = np.linalg.norm(outer_ring_nodes - target_pos, axis=1)
            closest_idx = np.argmin(dists) + outer_ring_start

            prev_idx = closest_idx
            beam_nodes.append(prev_idx)  # fixed outer ring node connected to beam

            for i in range(1, self.beam_nodes_per_beam):
                dist_out = self.radius + i * self.beam_length / (self.beam_nodes_per_beam - 1)
                x = dist_out * np.cos(angle)
                y = dist_out * np.sin(angle)
                z = 0.0
                new_node = np.array([x, y, z])
                self.nodes = np.vstack([self.nodes, new_node])
                new_idx = self.N
                beam_nodes.append(new_idx)
                beam_edges.append((prev_idx, new_idx))
                prev_idx = new_idx
                self.N += 1

        self.beam_edges = beam_edges
        self.edges.extend(beam_edges)

        for (i, j) in beam_edges:
            dist = np.linalg.norm(self.nodes[i] - self.nodes[j])
            self.rest_lengths = np.append(self.rest_lengths, dist)

        self.fixed_indices = list(set(beam_nodes))
        print(f"Fixed indices for beams: {self.fixed_indices}") """
    
    def _finalize_state(self):
        self.masses = np.full(self.N, self.m)
        self.state0 = np.concatenate((self.nodes.flatten(), np.zeros(3 * self.N)))

        self.adjacency_matrix = np.zeros((self.N, self.N))
        for i, j in self.edges:
            self.adjacency_matrix[i, j] = 1
            self.adjacency_matrix[j, i] = 1
        print(f"Adjacency matrix:\n{self.adjacency_matrix}")
        
    
    def compute_forces(self, positions, velocities):
        forces = np.zeros_like(positions)
        for i in range(self.N):
            forces[3*i:3*i+3] =  forces[3*i:3*i+3] + self.gravity.reshape(3) * float(self.masses[i])
        
        for idx, (i, j) in enumerate(self.edges):
            pos_i = positions[3*i:3*i+3]
            pos_j = positions[3*j:3*j+3]
            delta = pos_i - pos_j
            dist = np.linalg.norm(delta)
            if dist == 0:
                continue
            direction = delta / dist
            force_magnitude = -self.k * (dist - self.rest_lengths[idx])
            force = force_magnitude * direction
            forces[3*i:3*i+3] += force
            forces[3*j:3*j+3] -= force
        
        for i in range(self.N):
            vel = velocities[3*i:3*i+3]
            forces[3*i:3*i+3] -= self.damping * vel
        
        
        return forces
    
    def ode(self, t, state):
        positions = state[:3*self.N]
        velocities = state[3*self.N:]

        forces = self.compute_forces(positions, velocities)
        accelerations = forces / np.repeat(self.masses, 3)
        
        derivatives = np.zeros_like(state)
        derivatives[:3*self.N] = velocities
        derivatives[3*self.N:] = accelerations
        
    
        
        return derivatives


In [ ]:

def total_dynamics(N, m_drone, V_d, Gamma_i, masses, L, r, k, lambda_i, m_ball, v_ball, t_ball, theta_d, t_dead, t_second_fault, index_dead, index_second_fault, controller, cloth, log_file='force_log.csv'):
     # Initialize the log file
    if os.path.exists(log_file):
        os.remove(log_file)
    with open(log_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header = ['time'] + [f'f{i}_{axis}' for i in range(N) for axis in ['x', 'y', 'z']]
        writer.writerow(header)
    def dynamics(t, state):

        if np.abs(t-t_ball) < 1e-2 and flag_ball[0] == False:
            cloth.masses[cloth.central_index] = cloth.m + m_ball
            state[8*N + 3*cloth.N + 3*cloth.central_index + 2] = (cloth.m*state[8*N + 3*cloth.N + 3*cloth.central_index + 2] + m_ball*v_ball)/(cloth.m + m_ball)
            flag_ball[0] = True
            print(f"Ball added at time {t}")
        if np.abs(t-t_dead) < 1e-2 and flag_dead[0] == False:
            Fxi = controller.rho[index_dead]*np.cos(theta_d[index_dead])
            Fyi = controller.rho[index_dead]*np.sin(theta_d[index_dead])
            Fxi_right = controller.rho[index_dead-1]*np.cos(theta_d[index_dead-1]) + 1/2*Fxi
            Fyi_right = controller.rho[index_dead-1]*np.sin(theta_d[index_dead-1]) + 1/2*Fyi
            Fxi_left = controller.rho[index_dead+1]*np.cos(theta_d[index_dead+1]) + 1/2*Fxi
            Fyi_left = controller.rho[index_dead+1]*np.sin(theta_d[index_dead+1]) + 1/2*Fyi

            """theta_d[index_dead-1] = np.arctan2(Fyi_right, Fxi_right)
            theta_d[index_dead+1] = np.arctan2(Fyi_left, Fxi_left)
            controller.rho[index_dead-1] = np.linalg.norm([Fxi_right, Fyi_right])
            controller.rho[index_dead+1] = np.linalg.norm([Fxi_left, Fyi_left])"""
            for i in range(N):
                if i == index_dead:
                    continue
                elif i < index_dead:
                    theta_d[i] = 2*np.pi/(N-1)*i
                elif i > index_dead:
                    theta_d[i] = 2*np.pi/(N-1)*(i-1)

            L[index_dead, :] = 0
            L[:, index_dead] = 0
            
            flag_dead[0] = True
            print(f"Drone {index_dead} dead at time {t}")
        if np.abs(t-t_second_fault) < 1e-2 and flag_second_fault[0] == False:
            Fxi = controller.rho[index_second_fault]*np.cos(theta_d[index_second_fault])
            Fyi = controller.rho[index_second_fault]*np.sin(theta_d[index_second_fault])
            Fxi_right = controller.rho[index_second_fault-1]*np.cos(theta_d[index_second_fault-1]) + 1/2*Fxi
            Fyi_right = controller.rho[index_second_fault-1]*np.sin(theta_d[index_second_fault-1]) + 1/2*Fyi
            Fxi_left = controller.rho[index_second_fault+1]*np.cos(theta_d[index_second_fault+1]) + 1/2*Fxi
            Fyi_left = controller.rho[index_second_fault+1]*np.sin(theta_d[index_second_fault+1]) + 1/2*Fyi

            theta_d[index_second_fault-1] = np.arctan2(Fyi_right, Fxi_right)
            theta_d[index_second_fault+1] = np.arctan2(Fyi_left, Fxi_left)
            controller.rho[index_second_fault-1] = np.linalg.norm([Fxi_right, Fyi_right])
            controller.rho[index_second_fault+1] = np.linalg.norm([Fxi_left, Fyi_left])

            L[index_second_fault, :] = 0
            L[:, index_second_fault] = 0
            
            flag_second_fault[0] = True
            print(f"Drone {index_second_fault} dead at time {t}")

        print(f"t: {t:.4f}", end="\r", flush=True)

        drones_state = state[:8*N]
        drones_dynamics = create_multi_agent_dynamics_with_center(N, masses, L, controller, V_d, r, theta_d=theta_d, index_dead = index_dead, index_second_fault=index_second_fault, lambda_i = lambda_i)
        drones_derivatives = drones_dynamics(t, drones_state)

        cloth_state = state[8*N:]
        cloth_derivatives = cloth.ode(t, cloth_state)

        # Interface physics 
        #fixed_indices = [idx + 7*N for idx in cloth.fixed_indices]

        f = np.zeros((3, N))
        #Check che al primo drone corrisponda il primo punto nella lista
        for i, idx in enumerate(cloth.fixed_indices):
            x_i_pos = drones_state[6*i:6*i+3]

            a_i_pos = cloth_state[3*idx:3*idx+3]

            f_i = k*(x_i_pos - a_i_pos)
            f[:, i] = f_i.flatten()
            drones_derivatives[6*i+3:6*i+6] = drones_derivatives[6*i+3:6*i+6] - f_i/masses[i]
            cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] = cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] + f_i/cloth.m
            #drones_derivatives[N*i+3:N*i+6] = cloth_derivatives[3*cloth.N + 3*idx:3*cloth.N + 3*idx+3] #drones_derivatives[N*i+3:N*i+6] - f_i/masses[i]
            #drones_derivatives[&*i:N*i+3] = cloth_state[3*idx:3*idx+3]
        if flag_dead[0] == True:
            drones_derivatives[6*N + index_dead] = 0
        if flag_second_fault[0] == True:
            drones_derivatives[6*N + index_second_fault] = 0
        # Log time and f_i
        with open(log_file, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            flat_f = f.T.flatten()  # shape: (3*N,) in [f1x, f1y, f1z, f2x, ...]
            writer.writerow([t] + flat_f.tolist())

        return np.concatenate([drones_derivatives, cloth_derivatives])

    return dynamics


In [ ]:
def compute_laplacian_matrix(N, type='full'):
	L = np.zeros((N,N))
	if type == 'full':
		L = -np.ones((N,N)) + np.diag(N*np.ones(N))
	elif type == 'neighbors':
		L = np.diag(2*np.ones(N)) - np.diag(np.ones(N-1), k = 1) - np.diag(np.ones(N-1), k = -1)
		L[0, -1] = -1
		L[-1, 0] = -1
	return L

In [ ]:
def vd(t):
    if t<= 1.57*5:
        return np.array([2.0, 0.0, 0.0]).reshape(-1,1)
    elif t > 1.57*5 and t <= 9.42*5:
        return np.array([2.0 * np.sin(t), -2.0 * np.cos(t), 0.0]).reshape(-1,1)
    else:
        return np.array([0.0, 0.0, 0.0]).reshape(-1,1)





In [ ]:
N = 10

m_payload = 1.5
m_drone = 0.375 #kg
g = 9.81
k = 1000  # Spring constant for the cloth
lambda_i = 10.0


# Ball
m_ball = 10
v_ball = -1 #velocita finale
t_ball = 20 #10.0
flag_ball = [False]

# Reconfiguration
t_dead = 10 #20
index_dead = 2
flag_dead = [False]

# Second Fault, the graph becomes disconnected !
t_second_fault = 30
index_second_fault = 8
flag_second_fault = [False]
# Cloath
m_cloth = 1.5
num_rings = 5
points_per_ring = 20
num_cloth_points = (num_rings-1)*points_per_ring + 1 
m_cloth_per_point = m_cloth/num_cloth_points
cloth = CircularCloth(k=1000, damping=0.1, m=m_cloth_per_point, num_rings=num_rings, points_per_ring=points_per_ring, num_beams=N, central_index = 0)

V_d = lambda t: 0.0*(1-np.exp(-t))*np.array([2,1,0]).reshape(-1,1)#lambda t: vd(t) #
Gamma_i = np.array([[150, 0, 0], [0, 150, 0], [0, 0, 150]])
masses = np.ones((N,1))*m_drone
L = compute_laplacian_matrix(N, "neighbors") #np.array([[2, -1, -1], [-1, 1, 0], [-1, 0, 1]])

# Add a small random perturbation to the initial positions
np.random.seed(42)  # for reproducibility
perturbation = 0.0 * np.random.randn(3, len(cloth.fixed_indices))
r = np.column_stack([cloth.state0[3*i:3*i+3] for i in cloth.fixed_indices]) + perturbation
print(r)
#r = np.array([[0, 2.5, 0], [2.5*np.sin(np.pi/3), -2.5*np.cos(np.pi/3), 0], [-2.5*np.sin(np.pi/3), -2.5*np.cos(np.pi/3), 0]]).T

initial_condition_drone = np.zeros((6*(N)+N + N))

for i in range(N):
    initial_condition_drone[(i)*6: (i)*6+3] = r[:,i]

initial_condition_drone[-2*N:-N] = np.linspace((m_cloth-1)/N,(m_cloth+1)/N, N) #np.array([0.4, 0.5, 0.6]) #np.ones(3)*0.5
for idx, i in enumerate(range(-N, 0)):
    initial_condition_drone[i] = 0.0#np.arctan2(r[1,idx], r[0,idx])

rho_d = np.ones(N) * 25  # Initial rho values for each drone
theta_d = np.array([2*np.pi*i/N for i in range(N)])
controller = Controller(N, Gamma_i, V_d, r, masses, rho_d = rho_d)


initial_condition = np.concatenate([initial_condition_drone.flatten(), cloth.state0])

dynamics = total_dynamics(N, m_drone, V_d, Gamma_i, masses, L, r, k, lambda_i, m_ball, v_ball, t_ball, theta_d, t_dead, t_second_fault, index_dead, index_second_fault, controller, cloth)

T_sim = 40
t_eval = np.linspace(0, T_sim, 1000)
solution = solve_ivp(dynamics, (0, T_sim), initial_condition, t_eval=t_eval)



In [ ]:
import csv
import numpy as np

def dump_total_state_to_csv(solution, N, cloth, filename='total_state_dump.csv'):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)

        # --- HEADER ---
        headers = ['time']

        # Drone positions and velocities
        for i in range(N):
            headers += [
                f'drone_{i}_x', f'drone_{i}_y', f'drone_{i}_z',
                f'drone_{i}_vx', f'drone_{i}_vy', f'drone_{i}_vz'
            ]

        # Drone masses
        for i in range(N):
            headers += [f'drone_{i}_mass']

        for j in range(cloth.N):  # N = numero nodi
            headers += [
                f'cloth_{j}_x', f'cloth_{j}_y', f'cloth_{j}_z'
            ]

        for j in range(cloth.N):
            headers += [
                f'cloth_{j}_vx', f'cloth_{j}_vy', f'cloth_{j}_vz'
            ]

        writer.writerow(headers)
        print(headers)

        # --- DATA ROWS ---
        for t_idx in range(len(solution.t)):
            row = [solution.t[t_idx]]
            row += solution.y[:, t_idx].tolist()
            writer.writerow(row)

    print(f"✔ Dumped full simulation state to '{filename}'")

dump_total_state_to_csv(solution, N, cloth)

In [ ]:
sol = solution.y[8*N:, :]
time = solution.t

# Plot snapshots
num_plots = 10
indices = np.linspace(0, sol.shape[1]-1, num_plots).astype(int)

fig = plt.figure(figsize=(20, 10))
for k, idx in enumerate(indices):
    ax = fig.add_subplot(2, 5, k+1, projection='3d')
    positions = sol[:3*cloth.N, idx].reshape((cloth.N, 3))


    # Plot all cloth nodes
    ax.scatter(positions[:,0], positions[:,1], positions[:,2], c='blue', s=10)

    # Plot fixed nodes in red
    fixed_idx = cloth.fixed_indices
    if fixed_idx:
        fixed_positions = positions[fixed_idx]
        ax.scatter(fixed_positions[:,0], fixed_positions[:,1], fixed_positions[:,2], c='red', s=30)

    # Draw cloth edges (exclude beams)
    cloth_edge_set = set(cloth.edges) - set(cloth.beam_edges)
    for i, j in cloth_edge_set:
        p1 = positions[i]
        p2 = positions[j]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'k-', lw=0.7)

    # Draw beams in red
    for i, j in cloth.beam_edges:
        p1 = positions[i]
        p2 = positions[j]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'r-', lw=2)

    drone_positions = np.array([
        solution.y[6*i : 6*i+3, idx] for i in range(3)
    ])
    ax.scatter(drone_positions[:,0], drone_positions[:,1], drone_positions[:,2],
               c='green', s=50, marker='^', label='Drones')
               
    ax.set_title(f'Time = {time[idx]:.2f}s')
    ax.set_xlim(-cloth.radius*2, cloth.radius*2)
    ax.set_ylim(-cloth.radius*2, cloth.radius*2)
    ax.set_zlim(-1, 1)

    # Enable XY tickers and labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zticks([])  # Optionally still hide Z-axis ticks

plt.suptitle('Circular Cloth with Fixed Radial Beams Simulation', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
def extract_states(sol, N):
    states = sol.y
    T = states.shape[1]  # number of time steps

    positions = np.zeros((N, 3, T))  # includes center
    velocities = np.zeros((N, 3, T))
    M_values = np.zeros((N, T))
    theta_values = np.zeros((N, T))

    for i in range(N):
        idx = 6 * i
        positions[i] = states[idx : idx + 3]
        
        velocities[i] = states[idx + 3 : idx + 6]

    for i in range(N):
        M_values[i] = states[6 * (N) + i]
    for i in range(N):
        theta_values[i] = states[7 * (N) + i]

    

    return positions, velocities, M_values, theta_values


In [ ]:
positions, velocities, M_values, theta_values = extract_states(solution, N)

# Plot z-position of center and agents
import matplotlib.pyplot as plt

for i in range(N):
    plt.plot(solution.t, positions[i, 2], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('z-position')
plt.legend()
plt.show()



In [ ]:
# Estrai dati di posizione e tempo
time = solution.t
mask = (time >= 0) & (time <= T_sim)

positions = positions[:, :, mask]
time = time[mask]

colors = ['r', 'g', 'b', 'k', 'm', 'c', 'y']

# --- Plot x(t) ---
plt.figure(figsize=(10, 4))
for i in range(N):
    x = positions[i, 0, :]
    plt.plot(time, x, color=colors[i % len(colors)], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('x [m]')
plt.title('X Position vs Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Plot y(t) ---
plt.figure(figsize=(10, 4))
for i in range(N):
    y = positions[i, 1, :]
    plt.plot(time, y, color=colors[i % len(colors)], label=f'Agent {i}')
plt.xlabel('Time [s]')
plt.ylabel('y [m]')
plt.title('Y Position vs Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Extract Mc_i values

# Time vector from solve_ivp
t = solution.t

# Find indices where time is between 0 and 1
mask = (t >= 0) & (t <= T_sim)
t_filtered = t[mask]
M_filtered = M_values[:, mask]

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(t_filtered, M_filtered[i], label=f'$M_{{{i+1}}}(t)$')

plt.xlabel('Time [s]')
plt.ylabel('$M_i$ values')
plt.title('Adaptive Parameter $M_i$ from $t=0$ to $t=1$')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
t = solution.t

# Filter time and corresponding theta values
mask = (t >= 0) & (t <= T_sim)
t_filtered = t[mask]
theta_filtered = theta_values[:, mask]
# Unwrap first (operates along time axis for each drone)
"""theta_unwrapped = np.unwrap(theta_filtered, axis=1)

# Then wrap back to [0, 2π] to avoid discontinuities in the plot
theta_smooth = np.mod(theta_unwrapped, 2 * np.pi)

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(t_filtered, theta_smooth[i], label=f'$\\theta_{{{i+1}}}(t)$')

plt.xlabel('Time [s]')
plt.ylabel('$\\theta_i$ [rad]')
plt.title('Adaptive Parameter $\\theta_i$ (wrapped smoothly to $[0, 2\\pi]$)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()"""
# Step 1: Unwrap l'array originale (senza modulo)
theta_unwrapped = np.unwrap(theta_filtered, axis=1)

# Step 2: Ricalcolo relativo al primo valore di ogni drone
# (evita disallineamenti visuali)
theta_relative = theta_unwrapped - theta_unwrapped[:, [0]]

# Step 3: Ri-mappa nel range [0, 2π] solo per bellezza visiva
theta_wrapped_smooth = np.mod(theta_relative + theta_filtered[:, [0]], 2 * np.pi)

# Plot
plt.figure(figsize=(10, 6))
for i in range(N):
    plt.plot(t_filtered, theta_wrapped_smooth[i], label=f'$\\theta_{{{i+1}}}(t)$')

plt.xlabel('Time [s]')
plt.ylabel('$\\theta_i$ [rad]')
plt.title('Adaptive Parameter $\\theta_i$ (visually smoothed in $[0, 2\\pi]$)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume: N = 3 agents (excluding center), so 4 total

# Time range (optional): limit to 0–1 s if needed
mask = (solution.t >= 0) & (solution.t <= T_sim)
positions_xy = positions[:, :2, mask]  # shape: (N+1, 2, T)

# Plot XY trajectories
plt.figure(figsize=(8, 8))
colors = ['r', 'g', 'b', 'k']

for i in range(N):  # 0 = center, 1...N = agents
    x = positions_xy[i, 0, :]
    y = positions_xy[i, 1, :]
    plt.plot(x, y, color=colors[i % len(colors)], label=f'Agent {i}')
    plt.plot(x[0], y[0], 'o', color=colors[i % len(colors)])  # start point
    plt.plot(x[-1], y[-1], 'x', color=colors[i % len(colors)])  # end point

plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('XY Motion of Agents (t = 0 to 1 s)')
plt.grid(True)
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()



In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume positions_xyz shape: (N+1, 3, T)
# positions_xyz, N, and T_sim are already defined as per your code snippet

colors = ['red', 'green', 'blue', 'black']

fig = go.Figure()

for i in range(N):
    x = positions[i, 0, :]
    y = positions[i, 1, :]
    z = positions[i, 2, :]
    
    # Trajectory line
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='lines+markers',
        name=f'Agent {i}',
        line=dict(color=colors[i % len(colors)], width=4),
        marker=dict(size=4),
        marker_symbol='circle'
    ))
    
    # Start marker (bigger circle)
    fig.add_trace(go.Scatter3d(
        x=[x[0]], y=[y[0]], z=[z[0]],
        mode='markers',
        marker=dict(color=colors[i % len(colors)], size=8, symbol='circle'),
        name=f'Start Agent {i}',
        showlegend=False
    ))
    
    # End marker (cross)
    fig.add_trace(go.Scatter3d(
        x=[x[-1]], y=[y[-1]], z=[z[-1]],
        mode='markers',
        marker=dict(color=colors[i % len(colors)], size=8, symbol='x'),
        name=f'End Agent {i}',
        showlegend=False
    ))

fig.update_layout(
    title='3D Motion of Agents (t = 0 to 1 s)',
    scene=dict(
        xaxis_title='x [m]',
        yaxis_title='y [m]',
        zaxis_title='z [m]',
        aspectmode='data'
    ),
    legend=dict(itemsizing='constant')
)

fig.show()


In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume: positions.shape = (N, 3, T)
T = positions.shape[2]
colors = ['red', 'green', 'blue', 'black']

# Initial traces at t=0
data = []
for i in range(N):
    # Trail at t=0 is a single point
    data.append(go.Scatter3d(
        x=[positions[i, 0, 0]],
        y=[positions[i, 1, 0]],
        z=[positions[i, 2, 0]],
        mode='lines+markers',
        name=f'Agent {i}',
        line=dict(color=colors[i % len(colors)], width=4),
        marker=dict(size=4),
        showlegend=True
    ))

# Build frames
frames = []
for t in range(1, T):
    frame_data = []
    for i in range(N):
        # Add trajectory from start to current time
        frame_data.append(go.Scatter3d(
            x=positions[i, 0, :t+1],
            y=positions[i, 1, :t+1],
            z=positions[i, 2, :t+1],
            mode='lines+markers',
            line=dict(color=colors[i % len(colors)], width=4),
            marker=dict(size=4),
            name=f'Agent {i}',
            showlegend=False  # Keep legend only once
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

# Axis limits
x_range = [positions[:, 0, :].min(), positions[:, 0, :].max()]
y_range = [positions[:, 1, :].min(), positions[:, 1, :].max()]
z_range = [positions[:, 2, :].min(), positions[:, 2, :].max()]

# Layout
layout = go.Layout(
    title='Animated 3D Motion of Agents',
    scene=dict(
        xaxis=dict(title='x [m]', range=x_range),
        yaxis=dict(title='y [m]', range=y_range),
        zaxis=dict(title='z [m]', range=z_range),
        aspectmode='data',
        camera=dict(
            eye=dict(x=2.0, y=2.0, z=1.0)  # ← Customize this as needed
        )
    ),
    updatemenus=[dict(
        type='buttons',
        showactive=True,
        buttons=[
            dict(label='Play', method='animate', args=[None, {
                'frame': {'duration': 1000 / T, 'redraw': True},
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]),
            dict(label='Pause', method='animate', args=[[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }])
        ]
    )]
)

fig = go.Figure(data=data, layout=layout, frames=frames)
fig.show()

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume: positions (N, 3, T), cloth_positions (M, 3, T)
sol = solution.y[8*N:, :]

cloth_positions = sol[:3*cloth.N, :].reshape((cloth.N, 3, -1))

T = positions.shape[2]

# Camera settings
fixed_camera = dict(eye=dict(x=2.0, y=2.0, z=1.0)) 
aspect_ratio_ = dict(x=1, y=1, z=1)
# Axis limits
x_range = [min(positions[:, 0, :].min(), cloth_positions[:, 0, :].min()),
           max(positions[:, 0, :].max(), cloth_positions[:, 0, :].max())]
y_range = [min(positions[:, 1, :].min(), cloth_positions[:, 1, :].min()),
           max(positions[:, 1, :].max(), cloth_positions[:, 1, :].max())]
z_range = [min(positions[:, 2, :].min(), cloth_positions[:, 2, :].min()),
           max(positions[:, 2, :].max(), cloth_positions[:, 2, :].max()) + 1.5] # TODO:

colors = ['red', 'green', 'blue', 'black']

# Initial traces at t=0
data = []

# --- Drones ---
for i in range(N):
    data.append(go.Scatter3d(
        x=[positions[i, 0, 0]],
        y=[positions[i, 1, 0]],
        z=[positions[i, 2, 0]],
        mode='markers+text',
        name=f'Agent {i}',
        marker=dict(size=6, color=colors[i % len(colors)]),
        text=[f'{i}'],
        textposition='top center',
        showlegend=True
    ))

# --- Cloth Nodes (blue) ---
data.append(go.Scatter3d(
    x=cloth_positions[:, 0, 0],
    y=cloth_positions[:, 1, 0],
    z=cloth_positions[:, 2, 0],
    mode='markers',
    marker=dict(size=3, color='blue'),
    name='Cloth Nodes',
    showlegend=True
))

# --- Fixed Nodes (red) ---
fixed_idx = cloth.fixed_indices
if fixed_idx:
    fixed_positions = cloth_positions[fixed_idx, :, 0]
    data.append(go.Scatter3d(
        x=fixed_positions[:, 0],
        y=fixed_positions[:, 1],
        z=fixed_positions[:, 2],
        mode='markers',
        marker=dict(size=5, color='red'),
        name='Fixed Nodes',
        showlegend=True
    ))

# --- Cloth Edges ---
edge_lines = []
for i, j in cloth.edges:
    p1 = cloth_positions[i, :, 0]
    p2 = cloth_positions[j, :, 0]
    edge_lines.extend([p1, p2, [None, None, None]])
edge_lines = np.array(edge_lines).T

data.append(go.Scatter3d(
    x=edge_lines[0],
    y=edge_lines[1],
    z=edge_lines[2],
    mode='lines',
    line=dict(color='gray', width=1),
    name='Cloth Edges',
    showlegend=False
))

# --- Tempo iniziale (t = 0) ---
data.append(go.Scatter3d(
    x=[x_range[0]],
    y=[y_range[1]],
    z=[z_range[1]],
    mode='text',
    text=[f't = 0.00 s'],
    showlegend=False,
    textfont=dict(size=12, color='black')
))


# --- FRAMES ---
frames = []



# Palla: traiettoria verticale verso cloth.central_index
t_index = np.searchsorted(solution.t, t_ball)
z_impact = cloth_positions[cloth.central_index, 2, t_index]
x_impact = cloth_positions[cloth.central_index, 0, t_index]
y_impact = cloth_positions[cloth.central_index, 1, t_index]

from collections import deque

ball_positions = deque()
ball_positions.append([x_impact, y_impact, z_impact])  # t_index

# Before impact – prepend positions
z_curr = z_impact
for t_idx in range(t_index - 1, -1, -1):
    t_delta = solution.t[t_idx + 1] - solution.t[t_idx]
    z_curr -= v_ball * t_delta
    ball_positions.appendleft([x_impact, y_impact, z_curr])

# After impact – append positions
for t_idx in range(t_index + 1, len(solution.t)):
    z_post = cloth_positions[cloth.central_index, 2, t_idx]
    x_post = cloth_positions[cloth.central_index, 0, t_idx]
    y_post = cloth_positions[cloth.central_index, 1, t_idx]
    ball_positions.append([x_post, y_post, z_post])

ball_positions = np.array(ball_positions)

for t in range(1, T):
    frame_data = []

    # Drones
    for i in range(N):
        frame_data.append(go.Scatter3d(
            x=[positions[i, 0, t]],
            y=[positions[i, 1, t]],
            z=[positions[i, 2, t]],
            mode='markers',
            marker=dict(size=6, color=colors[i % len(colors)]),
            name=f'Agent {i}',
            showlegend=False
        ))

    # Cloth nodes (blue)
    frame_data.append(go.Scatter3d(
        x=cloth_positions[:, 0, t],
        y=cloth_positions[:, 1, t],
        z=cloth_positions[:, 2, t],
        mode='markers',
        marker=dict(size=3, color='blue'),
        name='Cloth Nodes',
        showlegend=False
    ))

    # --- Palla ---
    ball_xyz = ball_positions[t]
    frame_data.append(go.Scatter3d(
        x=[ball_xyz[0]],
        y=[ball_xyz[1]],
        z=[ball_xyz[2]],
        mode='markers',
        marker=dict(size=6, color='orange', symbol='circle'),
        name='Ball',
        showlegend=(t == 1)
    ))

    # Fixed nodes (red)
    if fixed_idx:
        fixed_positions = cloth_positions[fixed_idx, :, t]
        frame_data.append(go.Scatter3d(
            x=fixed_positions[:, 0],
            y=fixed_positions[:, 1],
            z=fixed_positions[:, 2],
            mode='markers',
            marker=dict(size=5, color='red'),
            name='Fixed Nodes',
            showlegend=False
        ))

    # Cloth edges (gray)
    edge_lines = []
    for i, j in cloth.edges:
        p1 = cloth_positions[i, :, t]
        p2 = cloth_positions[j, :, t]
        edge_lines.extend([p1, p2, [None, None, None]])
    edge_lines = np.array(edge_lines).T

    frame_data.append(go.Scatter3d(
        x=edge_lines[0],
        y=edge_lines[1],
        z=edge_lines[2],
        mode='lines',
        line=dict(color='gray', width=1),
        name='Cloth Edges',
        showlegend=False
    ))

    # Tempo corrente in alto
    frame_data.append(go.Scatter3d(
        x=[x_range[0]],
        y=[y_range[1]],
        z=[z_range[1]],
        mode='text',
        text=[f't = {solution.t[t]:.2f} s'],
        showlegend=False,
        textfont=dict(size=12, color='black')
    ))

    frames.append(go.Frame(
    data=frame_data,
    name=str(t),
    layout=go.Layout(
        scene=dict(
            xaxis=dict(range=x_range),
            yaxis=dict(range=y_range),
            zaxis=dict(range=z_range),
            camera=fixed_camera,
            aspectmode='manual',
            aspectratio=aspect_ratio_  # O adatta z in base alla tua scena
        ),
        annotations=[
            dict(
                text=f"<b>t = {solution.t[t]:.2f} s</b>",
                showarrow=False,
                x=0.5,
                y=1.08,
                xref="paper",
                yref="paper",
                font=dict(size=16, color="black"),
                align="center"
            )
        ]
    )
))




# Layout
layout = go.Layout(
    width=1000,  # 👈 Larghezza in pixel
    height=800,  # 👈 Altezza in pixel
    title='Animated 3D Motion: Drones and Cloth',
    scene=dict(
        xaxis=dict(title='x [m]', range=x_range),
        yaxis=dict(title='y [m]', range=y_range),
        zaxis=dict(title='z [m]', range=z_range),
        #aspectmode='data',
        aspectmode='manual',
        aspectratio=aspect_ratio_,  # O adatta z in base alla tua scena
        camera=fixed_camera  # Wide initial view
    ),
    updatemenus=[dict(
        type='buttons',
        showactive=True,
        buttons=[
            dict(label='Play', method='animate', args=[None, {
                'frame': {'duration': 10, 'redraw': True},  # ← Slower
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]),
            dict(label='Pause', method='animate', args=[[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }])
        ]
    )],
    annotations=[
        dict(
            text="<b>t = 0.00 s</b>",
            showarrow=False,
            x=0.5,
            y=1.08,
            xref="paper",
            yref="paper",
            font=dict(size=16, color="black"),
            align="center"
        )
    ]
)

# Create figure and show
fig = go.Figure(data=data, layout=layout, frames=frames)
fig.show()

fig.write_html("animated_drones_cloth.html")

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Assume: positions_xyz shape = (N+1, 3, T)
T = positions.shape[2]
colors = ['red', 'green', 'blue', 'black']

# Duration per frame in milliseconds
frame_duration_ms = 1 / T

# Initial frame at t=0
data = []
for i in range(N):
    data.append(go.Scatter3d(
        x=[positions[i, 0, 0]],
        y=[positions[i, 1, 0]],
        z=[positions[i, 2, 0]],
        mode='markers+text',
        marker=dict(size=6, color=colors[i % len(colors)]),
        name=f'Agent {i}',
        text=[f'{i}'],
        textposition='top center'
    ))

# Create frames (no trail, just current point)
frames = []
for t in range(1, T):
    frame_data = []
    for i in range(N):
        frame_data.append(go.Scatter3d(
            x=[positions[i, 0, t]],
            y=[positions[i, 1, t]],
            z=[positions[i, 2, t]],
            mode='markers+text',  # <- keep consistent
            marker=dict(size=6, color=colors[i % len(colors)]),
            text=[f'{i}'],
            textposition='top center',
            name=f'Agent {i}',
            showlegend=False
        ))
    frames.append(go.Frame(data=frame_data, name=str(t)))

# Axis limits
x_range = [-5,5]
y_range = [-5,5]
z_range = [-2,1]

# Layout
layout = go.Layout(
    title='3D Animated Motion of Agents (10s total)',
    scene=dict(
        xaxis=dict(title='x [m]', range=x_range),
        yaxis=dict(title='y [m]', range=y_range),
        zaxis=dict(title='z [m]', range=z_range),
        aspectmode='cube'
    ),
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        buttons=[
            dict(label='Play', method='animate', args=[None, {
                'frame': {'duration': frame_duration_ms, 'redraw': True},
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]),
            dict(label='Pause', method='animate', args=[[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }])
        ]
    )]
)

fig = go.Figure(data=data, layout=layout, frames=frames)
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Mask to filter time
mask = (solution.t >= 0) & (solution.t <= T_sim)
t_filtered = solution.t[mask]
velocities_filtered = velocities[:, :, mask]  # shape: (N+1, 3, T_filtered)

components = ['x', 'y', 'z']
colors = ['r', 'g', 'b', 'k', 'orange', 'purple', 'cyan']

# Plot vx, vy, vz over [0, 1] s
for dim in range(3):  # 0=x, 1=y, 2=z
    plt.figure(figsize=(10, 4))
    for i in range(N):
        plt.plot(t_filtered, velocities_filtered[i, dim, :], label=f'Agent {i}', color=colors[i % len(colors)])
    plt.title(f'Velocity in {components[dim]} direction (0–1 s)')
    plt.xlabel('Time [s]')
    plt.ylabel(f'$v_{components[dim]}$ [m/s]')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV log
df = pd.read_csv('control_log.csv')

# Extract time
time = df['time']

# Identify how many agents by parsing columns
agent_ids = sorted(set(col.split('_')[0] for col in df.columns if col != 'time'))

# Plot control inputs for each agent
for agent_id in agent_ids:
    plt.figure(figsize=(10, 4))
    plt.plot(time, df[f'{agent_id}_x'], label='x')
    plt.plot(time, df[f'{agent_id}_y'], label='y')
    plt.plot(time, df[f'{agent_id}_z'], label='z')
    plt.title(f'Control Input for {agent_id.upper()}')
    plt.xlabel('Time [s]')
    plt.ylabel('Control Force')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define V_d(t)
V_d = lambda t: (1 - np.exp(-t)) * np.array([2.0, 1.0, 0.0])

# Time vector
t_vals = np.linspace(0, 5, 500)
Vx = []
Vy = []
Vz = []

# Compute components over time
for t in t_vals:
    v = V_d(t)
    Vx.append(v[0])
    Vy.append(v[1])
    Vz.append(v[2])

# Plot
plt.figure(figsize=(10, 5))
plt.plot(t_vals, Vx, label='Vx(t)')
plt.plot(t_vals, Vy, label='Vy(t)')
plt.plot(t_vals, Vz, label='Vz(t)')
plt.title('Desired Velocity V_d(t)')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the logged data
df_control = pd.read_csv('force_log.csv')   # control forces
df_f_hat = pd.read_csv('f_hat_log.csv')         # estimated internal forces

# Extract time and number of agents
time = df_control['time']
#N = (df_control.shape[1] - 1) // 3
components = ['x', 'y', 'z']

# Plot each component for each agent
fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

for i, comp in enumerate(components):
    for agent in range(N):
        # Plot actual control forces
        print(time.shape, df_control[f'f{agent}_{comp}'].shape, df_f_hat[f'f{agent}_{comp}'].shape)
        axs[i].plot(time, df_control[f'f{agent}_{comp}'], label=f'Agent {agent} f_{comp} (control)', linestyle='-')
        # Plot estimated forces
        axs[i].plot(time, df_f_hat[f'f{agent}_{comp}'], label=f'Agent {agent} f_hat_{comp}', linestyle='--')
    
    axs[i].set_ylabel(f'$f_{comp}$ [N]')
    axs[i].grid(True)
    axs[i].legend(loc='upper right')

axs[2].set_xlabel('Time [s]')
fig.suptitle('Control Forces vs Estimated Internal Forces $f_i$ Over Time')
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the data
df_control = pd.read_csv('force_log.csv')
df_f_hat = pd.read_csv('f_hat_log.csv')

# Extract time and number of agents
time = df_control['time']
N = (df_control.shape[1] - 1) // 3
components = ['x', 'y', 'z']

# Compute 2-norm of the difference between control forces and estimated forces
for agent in range(N):
    diff_sq = np.zeros(len(time))
    for comp in components:
        f = df_control[f'f{agent}_{comp}']
        f_hat = df_f_hat[f'f{agent}_{comp}']
        diff_sq += (f - f_hat)**2

    norm = np.sqrt(diff_sq)

    # Create a separate plot for each agent
    plt.figure(figsize=(8, 4))
    plt.plot(time, norm, label=f'Agent {agent}')
    plt.title(f'2-Norm of Force Difference for Agent {agent}')
    plt.xlabel('Time [s]')
    plt.ylabel('$\\|f_i - \\hat{{f}}_i\\|_2$ [N]')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
